In [1]:
import json
import requests
from lxml import etree
import requests
import time
import os

In [2]:
s = requests.Session()
s.headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

In [3]:
with open('fund.ini','r') as f:
    list=[]
    for line in f.readlines():
        kv = line.strip().split()
        list.append(kv)

In [4]:
def str_rate(new,old):
    return format((new-old)/old*100,'.2f')+'%'

In [5]:
def get(code):
    url = 'https://fund.10jqka.com.cn/data/client/myfund/'+code
    res = s.get(url).json()['data'][0]
    
    name = res['name']
    net = float(res['net'])
    enddate = res['enddate']
    net1 = float(res['net1'])
    hqcode = res['hqcode']
    
    url = 'https://gz-fund.10jqka.com.cn/?module=api&controller=index&action=chart&info=vm_fd_'+hqcode
    #print(url)
    gz_all_raw = s.get(url).content.decode().split(';')[1:]
    #print(gz_all_raw)

    gz_all = []
    for i in gz_all_raw:
        gz_all.append(float(i.split(',')[-3]))
            
    return name+' '+code,gz_all,net,enddate,net1


In [6]:
def show(name,gz_all,net,enddate,net1):
    gz_now = gz_all[-1]
    gz_max = max(gz_all)
    gz_min = min(gz_all)
    
    gz_str = '盘中估值 '+str(gz_now)+' '+str_rate(gz_now,net1)
    jz_str = '单位净值('+enddate[5:]+') '+str(net)+' '+str_rate(net,net1)
    gz_max_min_str = '目前最高/低估值 '+str(gz_max)+'/'+str(gz_min)+' '+str_rate(gz_max,net1)+'/'+str_rate(gz_min,net1)

    if net < net1:
        print('T_T')
    else:
        print('^_^')
    print(name)
    print(gz_max_min_str)
    print(gz_str)
    print(jz_str)
    print()
     
def str_final(name,gz_all,net,enddate,net1,share):
    gz_now = gz_all[-1]
    gz_max = max(gz_all)
    gz_min = min(gz_all)
    amount_now = (gz_now-net1) * share
    
    gz_str = '盘中估值 '+str(gz_now)+' '+str_rate(gz_now,net1)
    jz_str = '单位净值('+enddate[5:]+') '+str(net)+' '+str_rate(net,net1)
    gz_max_min_str = '目前最高/低估值 '+str(gz_max)+'/'+str(gz_min)+' '+str_rate(gz_max,net1)+'/'+str_rate(gz_min,net1)
    zjz_str = '总净值 '+format(net*share,'.4f')+'元 ('+enddate+') '+ format(abs(gz_now-net1) * share,'.2f')
    
    if time.strftime('%Y-%m-%d') == enddate:
        die = net < net1
        zjz_str = '总净值 '+format(net*share,'.4f')+'元 ('+enddate+') '+ format((net-net1) * share,'.2f')
    else :
        die = gz_now < net
        zjz_str = '总净值 '+format(net*share,'.4f')+'元 ('+enddate+') '+ format((gz_now-net) * share,'.2f')
        
    if die:
        s = 'T_T\n'
    else:
        s = '^_^\n'
    return s + name+'\n' + gz_max_min_str+'\n' + gz_str+'\n' + jz_str+'\n'+zjz_str+'\n\n'

def wirte_front(fname,str):
    f = open(fname,'r')
    with open('tmp','w') as t:
        t.write(str)
        t.write(f.read())
    f.close()    
    t.close()
    os.remove(fname)
    os.rename('tmp',fname)

In [7]:
def main():
    amount_gz = 0
    amount_jz = 0
    amount_net1 = 0
    s = ''
    jz = True
    with open('fund.txt','w') as  f:
        for j in list:
            share = float(j[1])
            name,gz_all,net,enddate,net1 = get(j[0])
            amount_gz += gz_all[-1]*share
            amount_jz += net * share
            amount_net1 += net1 * share
            if not time.strftime('%Y-%m-%d') == enddate:
                jz = False
            fstr = str_final(name,gz_all,net,enddate,net1,share)
            print(fstr)
            f.write(fstr)
        
    amount_now = amount_jz
    if not jz and int(time.strftime('%H'))>=9:
        amount_now = amount_gz

    if jz:
        s += '净值更新了！'
    elif int(time.strftime('%H'))<9:
        s += '还没开盘！'
    elif int(time.strftime('%H'))>=15:
        s += '收盘了！'
    s += '(' + time.strftime('%Y-%m-%d')+')\n'
    
    
    if amount_now > amount_net1:
        s += '^_^\n涨了 '+format(amount_now-amount_net1,'.2f')+' 元'
    else:
        s += 'T_T\n跌了 '+format(amount_net1-amount_now,'.2f')+' 元'
    s += ' ('+enddate+')'
    if not jz:
        s += '*'
    s += '\n\n'
    print(s)
    wirte_front('fund.txt',s)

In [9]:
main()

^_^
泰康中证港股通大消费A 006786
目前最高/低估值 1.61528/1.57888 -0.58%/-2.82%
盘中估值 1.60373 -1.29%
单位净值(07-06) 1.5954 -1.80%
总净值 85.5932元 (2021-07-06) 0.45


^_^
鹏华钢铁 502023
目前最高/低估值 1.68647/1.64882 0.81%/-1.45%
盘中估值 1.66648 -0.39%
单位净值(07-06) 1.662 -0.66%
总净值 49.8766元 (2021-07-06) 0.13


^_^
广发中证全指家用电器指数A 005063
目前最高/低估值 1.46433/1.44305 0.85%/-0.62%
盘中估值 1.45742 0.37%
单位净值(07-06) 1.4565 0.31%
总净值 39.5003元 (2021-07-06) 0.02


T_T
创金合信科技成长股票A 005495
目前最高/低估值 2.12549/2.06108 0.95%/-2.11%
盘中估值 2.09812 -0.35%
单位净值(07-06) 2.1101 0.22%
总净值 30.4909元 (2021-07-06) -0.17


T_T
信诚新兴产业混合 000209
目前最高/低估值 4.19206/3.99457 2.42%/-2.40%
盘中估值 4.08178 -0.27%
单位净值(07-06) 4.118 0.61%
总净值 4.1180元 (2021-07-06) -0.04


还没开盘！(2021-07-07)
T_T
跌了 1.69 元 (2021-07-06)*


